In [20]:
import pandas as pd
import numpy as np
import warnings
import itertools
import os
from multiprocessing import Pool, cpu_count
from functools import partial
import heapq # 종료 시간 관리를 위한 힙큐

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

# =========================================================
# 1. 환경 및 시뮬레이터
# =========================================================
class UpbitMicrostructure:
    FEE_RATE = 0.0005
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        else: return 0.1
    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick
    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulatorGrid:
    def __init__(self):
        self.env = UpbitMicrostructure()
        
    def run_grid_test(self, df_signals, df_hour, tp_mult, sl_mult, hold_hours):
        # -------------------------------------------------------
        # [설정] 최대 동시 보유 개수 (슬롯)
        # -------------------------------------------------------
        MAX_SLOTS = 3 
        
        initial_equity = 1_000_000
        equity = initial_equity
        peak = equity
        max_dd = 0
        trades = 0
        wins = 0
        
        active_trades = [] 
        
        for idx, row in df_signals.iterrows():
            current_time = row['date']
            
            # 1. [정산] 현재 시간 기준으로 "이미 끝난" 거래들을 정리해서 돈(Equity)으로 환산
            while active_trades and active_trades[0][0] <= current_time:
                _, pnl, principal = heapq.heappop(active_trades)
                equity += (principal + pnl) # 원금 + 수익금 회수
                
                # MDD 갱신 (돈이 들어왔을 때만 계산)
                if equity > peak: peak = equity
                dd = (equity - peak) / peak * 100
                if dd < max_dd: max_dd = dd
            
            # 2. [진입 판단] 슬롯이 꽉 찼으면 진입 포기 (현금 부족)
            if len(active_trades) >= MAX_SLOTS:
                continue
            
            # 파산 방지
            if equity <= 5000: break

            locked_principal = sum([t[2] for t in active_trades])
            total_asset = equity + locked_principal
            
            # 1회 진입 금액 = 총 자산 / 슬롯 수
            bet_amount = total_asset / MAX_SLOTS
            
            # 만약 현재 현금(equity)이 배팅액보다 적으면 -> 있는 현금만 씀
            if equity < bet_amount:
                bet_amount = equity
            
            # 최소 주문 금액 필터
            entry_p_raw = row['ETH_Close']
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(entry_p_raw), True)
            
            if bet_amount < 5000: continue

            # 4. [진입 실행]
            qty = bet_amount / entry_p
            
            # 현금 차감 (수수료 포함)
            pos_krw = qty * entry_p
            equity -= pos_krw  # 일단 돈을 지갑에서 뺌 (Active로 이동)
            entry_fee = pos_krw * self.env.FEE_RATE
            equity -= entry_fee # 수수료는 즉시 차감
            
            # ATR 및 목표가 계산
            atr_val = row['ETH_ATR_84'] if row['ETH_ATR_84'] > 0 else entry_p * 0.01
            tp_price = entry_p + (atr_val * tp_mult)
            sl_price = entry_p - (atr_val * sl_mult)
            
            entry_time = current_time + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(hours=hold_hours)
            
            # 5. [미래 시뮬레이션] 이 포지션이 언제, 얼마에 끝날지 미리 계산
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_p = 0
            triggered = False
            actual_exit_time = end_time
            
            for _, h_row in hourly_data.iterrows():
                if h_row['low'] <= sl_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(sl_price), False)
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break 
                if h_row['high'] >= tp_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(tp_price), False)
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break
            
            if not triggered:
                last_close = hourly_data.iloc[-1]['close'] if not hourly_data.empty else entry_p
                exit_p = self.env.apply_slippage(self.env.adjust_to_tick(last_close), False)
            
            # 6. [결과 저장] 
            sell_amt = qty * exit_p
            exit_fee = sell_amt * self.env.FEE_RATE
            
            # 순수익(PnL) = 판 돈 - 수수료 - 원금
            trade_pnl = sell_amt - exit_fee - pos_krw
            
            # 활성 포지션 리스트에 등록
            heapq.heappush(active_trades, (actual_exit_time, trade_pnl, pos_krw))
            
            trades += 1
            if trade_pnl > 0: wins += 1
            
        # [최종 정산]
        while active_trades:
            _, pnl, principal = heapq.heappop(active_trades)
            equity += (principal + pnl)
        
        total_ret = (equity - initial_equity) / initial_equity * 100
        win_rate = (wins / trades * 100) if trades > 0 else 0
        
        return {
            'Return': total_ret,
            'MDD': max_dd,
            'Trades': trades,
            'WinRate': win_rate,
            'Params': f"TP:{tp_mult} / SL:{sl_mult} / Hold:{hold_hours}H / Slots:{MAX_SLOTS}"
        }


# [병렬 처리를 위한 래퍼 함수]
def run_single_simulation(params, simulator, df_signals, df_hour):
    tp, sl, hold = params
    res = simulator.run_grid_test(df_signals, df_hour, tp, sl, hold)
    res.update({'TP': tp, 'SL': sl, 'Hold': hold})
    return res

if __name__ == "__main__":
    print("📂 Loading Data...")
    try:
        df_4h = pd.read_csv("eth_4hour_all.csv")
        df_hour = pd.read_csv("eth_hour.csv")
    except FileNotFoundError:
        print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        exit()
    
    df_4h['date'] = pd.to_datetime(df_4h['date'])
    df_hour['datetime'] = pd.to_datetime(df_hour['datetime'])
    
    # 2020년 이후 데이터만 사용
    df_4h = df_4h[df_4h['date'] >= "2020-01-01"].sort_values('date')
    df_hour = df_hour[df_hour['datetime'] >= "2020-01-01"].sort_values('datetime')
    
    print("⚙️ Calculating Extra Filters (Force Index & ER & BTC Regime)...")
    
    # ------------------------------------------------------------------
    # 1. Force Index (거래량 실린 추세 확인)
    # ------------------------------------------------------------------
    df_4h['ETH_Force_Index_13'] = (df_4h['ETH_Close'].diff(1) * df_4h['ETH_Volume']).ewm(span=13).mean()
    
    # ------------------------------------------------------------------
    # 2. Efficiency Ratio (추세 효율성)
    # ------------------------------------------------------------------
    er_period = 10
    change = df_4h['ETH_Close'].diff(er_period).abs()
    volatility = df_4h['ETH_Close'].diff(1).abs().rolling(er_period).sum()
    df_4h['ETH_ER_10'] = change / (volatility + 1e-9) 

    if 'BTC_Close' in df_4h.columns:
        df_4h['BTC_MA_200'] = df_4h['BTC_Close'].rolling(1200).mean()
        cond_btc_regime = (df_4h['BTC_Close'] > df_4h['BTC_MA_200'])
    else:
        print("⚠️ BTC_Close 컬럼이 없습니다. BTC 필터 없이 진행합니다.")
        cond_btc_regime = True # 필터 없음

    # ------------------------------------------------------------------
    # [Signal 생성] BTC 필터를 포함한 추세추종 로직
    # ------------------------------------------------------------------
    
    # (1) 모멘텀: RSI가 55 이상 (상승세)
    cond_rsi_bull = (df_4h['ETH_RSI_84'] > 55)
    
    # (2) 온체인: 김프가 너무 낮지 않음
    cond_onchain_bull = (df_4h['ETH_KP_Zscore'] > -0.5)
    
    # (3) 퀄리티: 거래량 실림 + 추세 깔끔함
    cond_force = (df_4h['ETH_Force_Index_13'] > 0)
    cond_er = (df_4h['ETH_ER_10'] > 0.20)
    
    # ★ 최종 시그널 조합 (BTC 장기 상승장일 때만 매매)
    df_4h['signal'] = np.where(
        cond_btc_regime & cond_rsi_bull & cond_force & cond_er, 
        1, 0
    )
    
    df_signals = df_4h[df_4h['signal'] == 1].copy()
    print(f"✅ Signal Count (BTC Regime Filtered): {len(df_signals)}")
    
    if df_signals.empty:
        print("❌ No signals found. 조건을 완화해보세요.")
        exit()

    # -------------------------------------------------------
    # 🚀 병렬 처리 Grid Search 시작
    # -------------------------------------------------------
    sim = QuantSimulatorGrid()
    
    tp_list = [2.0, 2.5, 3.0, 3.5 ,4.0,4.5, 5.0,5.5, 6.0]  
    sl_list = [1.5, 2.0, 2.5, 3.0]       
    hold_list = [12, 24, 48, 72, 96, 120]    
    
    param_combinations = list(itertools.product(tp_list, sl_list, hold_list))
    total_comb = len(param_combinations)
    
    num_cores = os.cpu_count()
    if num_cores is None: num_cores = 4
    
    print(f"🚀 Starting Parallel Grid Search on {num_cores} cores ({total_comb} combinations)...")
    
    with Pool(processes=num_cores) as pool:
        runner = partial(run_single_simulation, simulator=sim, df_signals=df_signals, df_hour=df_hour)
        results = pool.map(runner, param_combinations)

    print("✅ Grid Search Completed!")

    # -------------------------------------------------------
    # 🏆 챔피언 선정 및 결과 출력
    # -------------------------------------------------------
    df_res = pd.DataFrame(results)
    
    if not df_res.empty:
        best_ret = df_res.loc[df_res['Return'].idxmax()]
        
        df_safe = df_res[df_res['Return'] > 0]
        if not df_safe.empty:
            best_safe = df_safe.loc[df_safe['MDD'].idxmax()]
        else:
            best_safe = None
        
        print("\n" + "="*60)
        print(f"🏆 [Revenue King] Highest Return")
        print(f"Params: TP {best_ret['TP']} / SL {best_ret['SL']} / Hold {best_ret['Hold']}H")
        print(f"Return: {best_ret['Return']:.1f}% | MDD: {best_ret['MDD']:.2f}% | Trades: {best_ret['Trades']}")
        print("-" * 60)
        
        if best_safe is not None:
            print(f"🛡️ [Safety King] Best MDD (Profit > 0)")
            print(f"Params: TP {best_safe['TP']} / SL {best_safe['SL']} / Hold {best_safe['Hold']}H")
            print(f"Return: {best_safe['Return']:.1f}% | MDD: {best_safe['MDD']:.2f}% | Trades: {best_safe['Trades']}")
        else:
            print("🛡️ [Safety King] 수익을 낸 조합이 없습니다.")
        print("=" * 60)
        
        print("\n📊 Top 5 Performing Combinations:")
        print(df_res.sort_values('Return', ascending=False).head(5)[['Params', 'Return', 'MDD', 'Trades']])
        
    else:
        print("❌ 결과가 없습니다.")



📂 Loading Data...
⚙️ Calculating Extra Filters (Force Index & ER & BTC Regime)...
✅ Signal Count (BTC Regime Filtered): 1528
🚀 Starting Parallel Grid Search on 88 cores (216 combinations)...
✅ Grid Search Completed!

🏆 [Revenue King] Highest Return
Params: TP 6.0 / SL 2.5 / Hold 120H
Return: 1635.5% | MDD: -71.65% | Trades: 393
------------------------------------------------------------
🛡️ [Safety King] Best MDD (Profit > 0)
Params: TP 5.5 / SL 2.0 / Hold 12H
Return: 314.3% | MDD: -70.47% | Trades: 968

📊 Top 5 Performing Combinations:
                                    Params   Return    MDD  Trades
209  TP:6.0 / SL:2.5 / Hold:120H / Slots:3 1,635.50 -71.65     393
203  TP:6.0 / SL:2.0 / Hold:120H / Slots:3 1,584.79 -73.09     412
215  TP:6.0 / SL:3.0 / Hold:120H / Slots:3 1,579.60 -72.22     381
191  TP:5.5 / SL:3.0 / Hold:120H / Slots:3 1,355.97 -72.22     395
167  TP:5.0 / SL:3.0 / Hold:120H / Slots:3 1,351.30 -72.22     410


In [23]:
import pandas as pd
import numpy as np
import warnings
import itertools
import os
from multiprocessing import Pool, cpu_count
from functools import partial
import heapq

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

# =========================================================
# 1. 환경 및 시뮬레이터
# =========================================================
class UpbitMicrostructure:
    FEE_RATE = 0.0005
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        else: return 0.1
    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick
    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulatorGrid:
    def __init__(self):
        self.env = UpbitMicrostructure()
        
    def run_grid_test(self, df_signals, df_hour, tp_mult, sl_mult, hold_hours):
        MAX_SLOTS = 3 
        
        initial_equity = 1_000_000
        equity = initial_equity
        peak = equity
        max_dd = 0
        trades = 0
        wins = 0
        
        active_trades = [] 
        
        for idx, row in df_signals.iterrows():
            current_time = row['date']
            
            while active_trades and active_trades[0][0] <= current_time:
                _, pnl, principal = heapq.heappop(active_trades)
                equity += (principal + pnl)
                if equity > peak: peak = equity
                dd = (equity - peak) / peak * 100
                if dd < max_dd: max_dd = dd
            
            if len(active_trades) >= MAX_SLOTS: continue
            if equity <= 5000: break

            locked_principal = sum([t[2] for t in active_trades])
            total_asset = equity + locked_principal
            bet_amount = total_asset / MAX_SLOTS
            if equity < bet_amount: bet_amount = equity
            
            entry_p_raw = row['ETH_Close']
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(entry_p_raw), True)
            
            if bet_amount < 5000: continue

            qty = bet_amount / entry_p
            pos_krw = qty * entry_p
            equity -= pos_krw
            entry_fee = pos_krw * self.env.FEE_RATE
            equity -= entry_fee
            
            atr_val = row['ETH_ATR_84'] if row['ETH_ATR_84'] > 0 else entry_p * 0.01
            
            calc_sl_dist = atr_val * sl_mult
            max_sl_dist = entry_p * 0.05  # 최대 손실 -5% 제한
            
            real_sl_dist = min(calc_sl_dist, max_sl_dist)
            
            tp_price = entry_p + (atr_val * tp_mult)
            sl_price = entry_p - real_sl_dist
            
            entry_time = current_time + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(hours=hold_hours)
            
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_p = 0
            triggered = False
            actual_exit_time = end_time
            
            for _, h_row in hourly_data.iterrows():
                if h_row['low'] <= sl_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(sl_price), False)
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break 
                if h_row['high'] >= tp_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(tp_price), False)
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break
            
            if not triggered:
                last_close = hourly_data.iloc[-1]['close'] if not hourly_data.empty else entry_p
                exit_p = self.env.apply_slippage(self.env.adjust_to_tick(last_close), False)
            
            sell_amt = qty * exit_p
            exit_fee = sell_amt * self.env.FEE_RATE
            trade_pnl = sell_amt - exit_fee - pos_krw
            
            heapq.heappush(active_trades, (actual_exit_time, trade_pnl, pos_krw))
            
            trades += 1
            if trade_pnl > 0: wins += 1
            
        while active_trades:
            _, pnl, principal = heapq.heappop(active_trades)
            equity += (principal + pnl)
        
        total_ret = (equity - initial_equity) / initial_equity * 100
        win_rate = (wins / trades * 100) if trades > 0 else 0
        
        return {
            'Return': total_ret,
            'MDD': max_dd,
            'Trades': trades,
            'WinRate': win_rate,
            'Params': f"TP:{tp_mult} / SL:{sl_mult} / Hold:{hold_hours}H / Slots:{MAX_SLOTS}"
        }

def run_single_simulation(params, simulator, df_signals, df_hour):
    tp, sl, hold = params
    res = simulator.run_grid_test(df_signals, df_hour, tp, sl, hold)
    res.update({'TP': tp, 'SL': sl, 'Hold': hold})
    return res

if __name__ == "__main__":
    print("📂 Loading Data...")
    try:
        df_4h = pd.read_csv("eth_4hour_all.csv")
        df_hour = pd.read_csv("eth_hour.csv")
    except FileNotFoundError:
        print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        exit()
    
    df_4h['date'] = pd.to_datetime(df_4h['date'])
    df_hour['datetime'] = pd.to_datetime(df_hour['datetime'])
    df_4h = df_4h[df_4h['date'] >= "2020-01-01"].sort_values('date')
    df_hour = df_hour[df_hour['datetime'] >= "2020-01-01"].sort_values('datetime')
    
    print("⚙️ Calculating Indicators...")
    
    # 1. Force Index
    df_4h['ETH_Force_Index_13'] = (df_4h['ETH_Close'].diff(1) * df_4h['ETH_Volume']).ewm(span=13).mean()
    
    # 2. Efficiency Ratio
    er_period = 10
    change = df_4h['ETH_Close'].diff(er_period).abs()
    volatility = df_4h['ETH_Close'].diff(1).abs().rolling(er_period).sum()
    df_4h['ETH_ER_10'] = change / (volatility + 1e-9) 
    
    # 3. NATR (Normalized ATR): 변동성 % 지표
    # ATR을 가격으로 나눠서 "현재 변동성이 몇 %인가"를 계산
    df_4h['ETH_NATR'] = (df_4h['ETH_ATR_84'] / df_4h['ETH_Close']) * 100

    # 4. BTC Regime
    if 'BTC_Close' in df_4h.columns:
        df_4h['BTC_MA_200'] = df_4h['BTC_Close'].rolling(1200).mean()
        cond_btc_regime = (df_4h['BTC_Close'] > df_4h['BTC_MA_200'])
    else:
        cond_btc_regime = True

    # ------------------------------------------------------------------
    # [Signal 생성] 변동성 필터(NATR) 추가
    # ------------------------------------------------------------------
    
    cond_rsi_bull = (df_4h['ETH_RSI_84'] > 55)
    cond_onchain_bull = (df_4h['ETH_KP_Zscore'] > -0.5)
    cond_force = (df_4h['ETH_Force_Index_13'] > 0)
    cond_er = (df_4h['ETH_ER_10'] > 0.20)
    
    # ★ [신규 필터] 시장이 너무 미쳐 날뛸 때는 진입 금지
    # NATR이 4% 이상이면(4시간에 4% 위아래로 움직이면) 너무 위험함
    cond_not_crazy = (df_4h['ETH_NATR'] < 4.0)
    
    df_4h['signal'] = np.where(
        cond_btc_regime & cond_rsi_bull & cond_force & cond_er & cond_not_crazy, 
        1, 0
    )
    
    df_signals = df_4h[df_4h['signal'] == 1].copy()
    print(f"✅ Signal Count (Safe Mode): {len(df_signals)}")
    
    if df_signals.empty:
        print("❌ No signals found.")
        exit()

    sim = QuantSimulatorGrid()
    
    # 기존 파라미터 유지 (이제 SL이 5%로 제한되므로 안전함)
    tp_list = [4.0, 5.0, 6.0]
    sl_list = [2.0, 2.5, 3.0]
    hold_list = [72, 96, 120]
    
    param_combinations = list(itertools.product(tp_list, sl_list, hold_list))
    num_cores = os.cpu_count() or 4
    
    print(f"🚀 Starting Parallel Grid Search on {num_cores} cores...")
    
    with Pool(processes=num_cores) as pool:
        runner = partial(run_single_simulation, simulator=sim, df_signals=df_signals, df_hour=df_hour)
        results = pool.map(runner, param_combinations)

    print("✅ Grid Search Completed!")

    df_res = pd.DataFrame(results)
    
    if not df_res.empty:
        best_ret = df_res.loc[df_res['Return'].idxmax()]
        df_safe = df_res[df_res['Return'] > 0]
        best_safe = df_safe.loc[df_safe['MDD'].idxmax()] if not df_safe.empty else None
        
        print("\n" + "="*60)
        print(f"🏆 [Revenue King]")
        print(f"Params: {best_ret['Params']}")
        print(f"Return: {best_ret['Return']:.1f}% | MDD: {best_ret['MDD']:.2f}% | Trades: {best_ret['Trades']}")
        print("-" * 60)
        
        if best_safe is not None:
            print(f"🛡️ [Safety King]")
            print(f"Params: {best_safe['Params']}")
            print(f"Return: {best_safe['Return']:.1f}% | MDD: {best_safe['MDD']:.2f}% | Trades: {best_safe['Trades']}")
        
        print("\n📊 Top 5 Combinations:")
        print(df_res.sort_values('Return', ascending=False).head(5)[['Params', 'Return', 'MDD', 'Trades']])



📂 Loading Data...
⚙️ Calculating Indicators...
✅ Signal Count (Safe Mode): 1500
🚀 Starting Parallel Grid Search on 88 cores...
✅ Grid Search Completed!

🏆 [Revenue King]
Params: TP:6.0 / SL:3.0 / Hold:120H / Slots:3
Return: 1482.6% | MDD: -71.89% | Trades: 386
------------------------------------------------------------
🛡️ [Safety King]
Params: TP:5.0 / SL:2.0 / Hold:72H / Slots:3
Return: 784.2% | MDD: -70.97% | Trades: 517

📊 Top 5 Combinations:
                                   Params   Return    MDD  Trades
26  TP:6.0 / SL:3.0 / Hold:120H / Slots:3 1,482.57 -71.89     386
23  TP:6.0 / SL:2.5 / Hold:120H / Slots:3 1,456.83 -71.62     393
20  TP:6.0 / SL:2.0 / Hold:120H / Slots:3 1,396.79 -73.09     406
19   TP:6.0 / SL:2.0 / Hold:96H / Slots:3 1,187.95 -74.23     439
22   TP:6.0 / SL:2.5 / Hold:96H / Slots:3 1,159.19 -73.40     428


In [30]:
import pandas as pd
import numpy as np
import warnings
import itertools
import os
from multiprocessing import Pool, cpu_count
from functools import partial
import heapq

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

# =========================================================
# 1. 환경 및 시뮬레이터
# =========================================================
class UpbitMicrostructure:
    FEE_RATE = 0.0005
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        else: return 0.1
    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick
    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulatorGrid:
    def __init__(self):
        self.env = UpbitMicrostructure()
        
    def run_grid_test(self, df_signals, df_hour, tp_mult, sl_mult, hold_hours):
        MAX_SLOTS = 3 
        
        initial_equity = 1_000_000
        equity = initial_equity
        peak = equity
        max_dd = 0
        trades = 0
        wins = 0
        
        active_trades = [] 
        
        for idx, row in df_signals.iterrows():
            current_time = row['date']
            
            while active_trades and active_trades[0][0] <= current_time:
                _, pnl, principal = heapq.heappop(active_trades)
                equity += (principal + pnl)
                if equity > peak: peak = equity
                dd = (equity - peak) / peak * 100
                if dd < max_dd: max_dd = dd
            
            if len(active_trades) >= MAX_SLOTS: continue
            if equity <= 5000: break

            locked_principal = sum([t[2] for t in active_trades])
            total_asset = equity + locked_principal
            bet_amount = total_asset / MAX_SLOTS
            if equity < bet_amount: bet_amount = equity
            
            entry_p_raw = row['ETH_Close']
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(entry_p_raw), True)
            
            if bet_amount < 5000: continue

            qty = bet_amount / entry_p
            pos_krw = qty * entry_p
            equity -= pos_krw
            entry_fee = pos_krw * self.env.FEE_RATE
            equity -= entry_fee
            
            atr_val = row['ETH_ATR_84'] if row['ETH_ATR_84'] > 0 else entry_p * 0.01
            
            calc_sl_dist = atr_val * sl_mult
            max_sl_dist = entry_p * 0.05
            
            real_sl_dist = min(calc_sl_dist, max_sl_dist)
            
            tp_price = entry_p + (atr_val * tp_mult)
            sl_price = entry_p - real_sl_dist
            
            entry_time = current_time + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(hours=hold_hours)
            
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_p = 0
            triggered = False
            actual_exit_time = end_time
            
            for _, h_row in hourly_data.iterrows():
                # -------------------------------------------------------
                # 🚨 [수정된 로직] 현실적인 손절 체결
                # -------------------------------------------------------
                
                # 1. 손절(SL) 체크
                if h_row['low'] <= sl_price:
                    # Case A: 시작부터 갭락(Gap-Down)으로 SL 아래에서 시작함
                    if h_row['open'] <= sl_price:
                        # 어쩔 수 없이 시가(Open)에 던짐
                        exit_p = self.env.apply_slippage(h_row['open'], False)
                    
                    # Case B: 장중에 흐르면서 SL을 건드림 (일반적 상황)
                    else:
                        # 손절가(SL)에 정확히 체결된 것으로 처리
                        exit_p = self.env.apply_slippage(sl_price, False)
                        
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break 
                
                # 2. 익절(TP) 체크
                if h_row['high'] >= tp_price:
                    # 시작부터 갭상(Gap-Up)이면 시가에 팔고, 아니면 목표가에 팖
                    price_to_sell = h_row['open'] if h_row['open'] >= tp_price else tp_price
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(price_to_sell), False)
                    triggered = True
                    actual_exit_time = h_row['datetime']
                    break
            
            if not triggered:
                last_close = hourly_data.iloc[-1]['close'] if not hourly_data.empty else entry_p
                exit_p = self.env.apply_slippage(self.env.adjust_to_tick(last_close), False)
            
            sell_amt = qty * exit_p
            exit_fee = sell_amt * self.env.FEE_RATE
            trade_pnl = sell_amt - exit_fee - pos_krw
            
            heapq.heappush(active_trades, (actual_exit_time, trade_pnl, pos_krw))
            
            trades += 1
            if trade_pnl > 0: wins += 1
            
        while active_trades:
            _, pnl, principal = heapq.heappop(active_trades)
            equity += (principal + pnl)
        
        total_ret = (equity - initial_equity) / initial_equity * 100
        win_rate = (wins / trades * 100) if trades > 0 else 0
        
        return {
            'Return': total_ret,
            'MDD': max_dd,
            'Trades': trades,
            'WinRate': win_rate,
            'Params': f"TP:{tp_mult} / SL:{sl_mult} / Hold:{hold_hours}H / Slots:{MAX_SLOTS}"
        }


def run_single_simulation(params, simulator, df_signals, df_hour):
    tp, sl, hold = params
    res = simulator.run_grid_test(df_signals, df_hour, tp, sl, hold)
    res.update({'TP': tp, 'SL': sl, 'Hold': hold})
    return res

if __name__ == "__main__":
    print("📂 Loading Data...")
    try:
        df_4h = pd.read_csv("eth_4hour_all.csv")
        df_hour = pd.read_csv("eth_hour.csv")
    except FileNotFoundError:
        print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        exit()
    
    df_4h['date'] = pd.to_datetime(df_4h['date'])
    df_hour['datetime'] = pd.to_datetime(df_hour['datetime'])
    df_4h = df_4h[df_4h['date'] >= "2020-01-01"].sort_values('date')
    df_hour = df_hour[df_hour['datetime'] >= "2020-01-01"].sort_values('datetime')
    
    print("⚙️ Calculating Indicators & Safe Filter System...")
    
    # 1. Force Index
    df_4h['ETH_Force_Index_13'] = (df_4h['ETH_Close'].diff(1) * df_4h['ETH_Volume']).ewm(span=13).mean()
    
    # 2. Efficiency Ratio
    er_period = 10
    change = df_4h['ETH_Close'].diff(er_period).abs()
    volatility = df_4h['ETH_Close'].diff(1).abs().rolling(er_period).sum()
    df_4h['ETH_ER_10'] = change / (volatility + 1e-9) 
    
    # ------------------------------------------------------------------
    # 🚨 [수정] 필터 로직 (Boolean Indexing으로 확실하게 적용)
    # ------------------------------------------------------------------
    
    # 1. Base Signal (공격)
    cond_entry = (
        (df_4h['ETH_RSI_84'] > 55) &
        (df_4h['ETH_Force_Index_13'] > 0) &
        (df_4h['ETH_ER_10'] > 0.20)
    )
    
    # 2. Defense Filter (방어)
    # BTC 20일선 위에 있고 & 최근 24시간 급락 없고
    if 'BTC_Close' in df_4h.columns:
        df_4h['BTC_Ret_24h'] = df_4h['BTC_Close'].pct_change(6)
        # 20일선 (4시간봉 120개)
        df_4h['BTC_MA_20'] = df_4h['BTC_Close'].rolling(120).mean()
        
        cond_defense = (
            (df_4h['BTC_Close'] > df_4h['BTC_MA_20']) &  # 추세
            (df_4h['BTC_Ret_24h'] > -0.03)              # 급락 회피 (-3%보다 커야 함)
        )
    else:
        print("⚠️ BTC 데이터 없음. 방어 필터 비활성화.")
        cond_defense = True
        
    # 3. ETH Drawdown Filter (고점 대비 -5% 이내)
    df_4h['Max_7d'] = df_4h['ETH_Close'].rolling(42).max()
    cond_dd = ((df_4h['ETH_Close'] / df_4h['Max_7d'] - 1) > -0.05)
    
    # 4. Sentiment (김프)
    cond_sentiment = (df_4h['ETH_KP_Zscore'] > -2.0)

    # ★ 최종 시그널 (Boolean Indexing)
    # 0으로 초기화 후 조건 만족하는 곳만 1
    df_4h['signal'] = 0
    final_mask = (cond_entry & cond_defense & cond_dd & cond_sentiment)
    df_4h.loc[final_mask, 'signal'] = 1
    
    df_signals = df_4h[df_4h['signal'] == 1].copy()
    print(f"✅ Signal Count (Safe Filtered): {len(df_signals)}")
    
    if df_signals.empty:
        print("❌ No signals found. 필터가 너무 빡빡할 수 있습니다.")
        exit()

    sim = QuantSimulatorGrid()
    
    tp_list = [4.0, 5.0, 6.0]
    sl_list = [2.0, 2.5, 3.0]
    hold_list = [72, 96, 120]
    
    param_combinations = list(itertools.product(tp_list, sl_list, hold_list))
    num_cores = os.cpu_count() or 4
    
    print(f"🚀 Starting Parallel Grid Search on {num_cores} cores...")
    
    with Pool(processes=num_cores) as pool:
        runner = partial(run_single_simulation, simulator=sim, df_signals=df_signals, df_hour=df_hour)
        results = pool.map(runner, param_combinations)

    print("✅ Grid Search Completed!")

    df_res = pd.DataFrame(results)
    
    if not df_res.empty:
        best_ret = df_res.loc[df_res['Return'].idxmax()]
        df_safe = df_res[df_res['Return'] > 0]
        best_safe = df_safe.loc[df_safe['MDD'].idxmax()] if not df_safe.empty else None
        
        print("\n" + "="*60)
        print(f"🏆 [Revenue King]")
        print(f"Params: {best_ret['Params']}")
        print(f"Return: {best_ret['Return']:.1f}% | MDD: {best_ret['MDD']:.2f}% | Trades: {best_ret['Trades']}")
        print("-" * 60)
        
        if best_safe is not None:
            print(f"🛡️ [Safety King]")
            print(f"Params: {best_safe['Params']}")
            print(f"Return: {best_safe['Return']:.1f}% | MDD: {best_safe['MDD']:.2f}% | Trades: {best_safe['Trades']}")
        
        print("\n📊 Top 5 Combinations:")
        print(df_res.sort_values('Return', ascending=False).head(5)[['Params', 'Return', 'MDD', 'Trades']])


📂 Loading Data...
⚙️ Calculating Indicators & Safe Filter System...
✅ Signal Count (Safe Filtered): 1643
🚀 Starting Parallel Grid Search on 88 cores...
✅ Grid Search Completed!

🏆 [Revenue King]
Params: TP:6.0 / SL:3.0 / Hold:120H / Slots:3
Return: 1418.0% | MDD: -72.78% | Trades: 456
------------------------------------------------------------
🛡️ [Safety King]
Params: TP:5.0 / SL:2.0 / Hold:120H / Slots:3
Return: 813.3% | MDD: -72.09% | Trades: 519

📊 Top 5 Combinations:
                                   Params   Return    MDD  Trades
26  TP:6.0 / SL:3.0 / Hold:120H / Slots:3 1,417.97 -72.78     456
23  TP:6.0 / SL:2.5 / Hold:120H / Slots:3 1,197.75 -72.68     467
20  TP:6.0 / SL:2.0 / Hold:120H / Slots:3 1,131.02 -73.21     482
25   TP:6.0 / SL:3.0 / Hold:96H / Slots:3 1,117.01 -72.47     494
19   TP:6.0 / SL:2.0 / Hold:96H / Slots:3 1,025.39 -73.80     516


In [39]:
new_hour = pd.read_csv("eth_hour.csv")
new_hour['datetime'] = pd.to_datetime(new_hour['datetime'])

mask = (new_hour['datetime'] >= "2024-12-03 20:00") & (new_hour['datetime'] <= "2024-12-03 23:00")
print(new_hour.loc[mask, ['datetime','open','high','low','close']])


                 datetime         open         high          low        close
43107 2024-12-03 20:00:00 5,045,000.00 5,069,000.00 5,039,000.00 5,043,000.00
43108 2024-12-03 21:00:00 5,041,000.00 5,065,000.00 5,028,000.00 5,059,000.00
43109 2024-12-03 22:00:00 5,059,000.00 5,078,000.00 3,120,000.00 3,820,000.00
43110 2024-12-03 23:00:00 3,820,000.00 4,990,000.00 3,820,000.00 4,747,000.00


In [38]:
import pandas as pd
import numpy as np

# === 설정 ===
MASTER_PATH = "eth_4hour_raw_price.csv"

UPBIT_TICKERS = {
    'KRW-BTC': ('BTC', 'BTC'), 'KRW-ETH': ('ETH', 'ETH'), 'KRW-XRP': ('XRP', 'XRP'),
    'KRW-SOL': ('SOL', 'SOL'), 'KRW-ADA': ('ADA', 'ADA'), 'KRW-DOGE': ('DOGE', 'DOGE'),
    'KRW-AVAX': ('AVAX', 'AVAX'), 'KRW-DOT': ('DOT', 'DOT')
}

df = pd.read_csv(MASTER_PATH)

# ==========================================
# 1) 컬럼 존재 여부 체크
# ==========================================

required_cols = []
for short, base in UPBIT_TICKERS.values():
    required_cols += [
        f"{short}_Open", f"{short}_High", f"{short}_Low", f"{short}_Close",
        f"{short}_Bin_Open", f"{short}_Bin_High", f"{short}_Bin_Low", f"{short}_Bin_Close"
    ]

missing = [col for col in required_cols if col not in df.columns]

print("\n=== 1) 누락된 컬럼 체크 ===")
if missing:
    print("❌ 누락된 컬럼:", missing)
else:
    print("✅ 모든 가격 관련 컬럼 존재")


# ==========================================
# 2) 결측치·음수·0 체크
# ==========================================

print("\n=== 2) 가격 데이터 결측치 / 음수 / 0 체크 ===")
for short, base in UPBIT_TICKERS.values():
    cols = [
        f"{short}_Open", f"{short}_High", f"{short}_Low", f"{short}_Close",
        f"{short}_Bin_Open", f"{short}_Bin_High", f"{short}_Bin_Low", f"{short}_Bin_Close"
    ]
    for col in cols:
        if col not in df.columns:
            continue
        
        null_cnt = df[col].isna().sum()
        neg_cnt = (df[col] < 0).sum()
        zero_cnt = (df[col] == 0).sum()

        if null_cnt or neg_cnt or zero_cnt:
            print(f"⚠️ {col}: NaN={null_cnt}, 음수={neg_cnt}, 0={zero_cnt}")
        else:
            print(f"✅ {col}: 정상")


# ==========================================
# 3) UPBIT vs BINANCE 가격 비교 (이상치 감지)
# ==========================================

print("\n=== 3) 업비트-바이낸스 가격 비율 체크 ===")
for short, base in UPBIT_TICKERS.values():
    up = df[f"{short}_Close"]
    bn = df[f"{short}_Bin_Close"]

    # 상장 이후 구간만 사용 (0 제외)
    mask = (up > 0) & (bn > 0)

    if mask.sum() == 0:
        print(f"ℹ️ {short}: 상장 이전 구간만 존재 (비교 불가)")
        continue

    ratio = up[mask] / bn[mask]
    abnormal = (ratio > 1.2) | (ratio < 0.8)

    if abnormal.sum() > 0:
        print(f"⚠️ {short}: 상장 이후 가격 비율 이상치 {abnormal.sum()}개")
    else:
        print(f"✅ {short}: 상장 이후 가격 정합 OK")


# ==========================================
# 4) 타임스탬프 중복 / 정렬 체크
# ==========================================

print("\n=== 4) 타임스탬프 체크 ===")
if "date" in df.columns:
    dup = df["date"].duplicated().sum()
    print(f"중복된 날짜: {dup}")

    sorted_flag = df["date"].is_monotonic_increasing
    print("시간 정렬 여부:", "정상" if sorted_flag else "❌ 비정상")
else:
    print("⚠️ date 컬럼 없음")




=== 1) 누락된 컬럼 체크 ===
✅ 모든 가격 관련 컬럼 존재

=== 2) 가격 데이터 결측치 / 음수 / 0 체크 ===
✅ BTC_Open: 정상
✅ BTC_High: 정상
✅ BTC_Low: 정상
✅ BTC_Close: 정상
✅ BTC_Bin_Open: 정상
✅ BTC_Bin_High: 정상
✅ BTC_Bin_Low: 정상
✅ BTC_Bin_Close: 정상
✅ ETH_Open: 정상
✅ ETH_High: 정상
✅ ETH_Low: 정상
✅ ETH_Close: 정상
✅ ETH_Bin_Open: 정상
✅ ETH_Bin_High: 정상
✅ ETH_Bin_Low: 정상
✅ ETH_Bin_Close: 정상
✅ XRP_Open: 정상
✅ XRP_High: 정상
✅ XRP_Low: 정상
✅ XRP_Close: 정상
⚠️ XRP_Bin_Open: NaN=0, 음수=0, 0=1319
⚠️ XRP_Bin_High: NaN=0, 음수=0, 0=1319
⚠️ XRP_Bin_Low: NaN=0, 음수=0, 0=1319
⚠️ XRP_Bin_Close: NaN=0, 음수=0, 0=1319
⚠️ SOL_Open: NaN=0, 음수=0, 0=8877
⚠️ SOL_High: NaN=0, 음수=0, 0=8877
⚠️ SOL_Low: NaN=0, 음수=0, 0=8877
⚠️ SOL_Close: NaN=0, 음수=0, 0=8877
⚠️ SOL_Bin_Open: NaN=0, 음수=0, 0=6297
⚠️ SOL_Bin_High: NaN=0, 음수=0, 0=6297
⚠️ SOL_Bin_Low: NaN=0, 음수=0, 0=6297
⚠️ SOL_Bin_Close: NaN=0, 음수=0, 0=6297
⚠️ ADA_Open: NaN=0, 음수=0, 0=60
⚠️ ADA_High: NaN=0, 음수=0, 0=60
⚠️ ADA_Low: NaN=0, 음수=0, 0=60
⚠️ ADA_Close: NaN=0, 음수=0, 0=60
⚠️ ADA_Bin_Open: NaN=0, 음수=0, 0=1216
⚠️ AD

In [46]:
import pandas as pd
import numpy as np
import warnings
import itertools
import os
from multiprocessing import Pool, cpu_count
from functools import partial
import heapq

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

# =========================================================
# 1. 환경 및 시뮬레이터 클래스
# =========================================================
class UpbitMicrostructure:
    FEE_RATE = 0.0005
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        else: return 0.1
    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick
    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulatorGrid:
    def __init__(self):
        self.env = UpbitMicrostructure()

    def run_grid_test(self, df_signals, df_hour, tp_mult, sl_mult, hold_hours):
        
        # --- 1. 모든 거래와 이벤트를 미리 계산 ---
        potential_trades = []
        df_hour_indexed = df_hour.set_index('datetime')

        for _, row in df_signals.iterrows():
            entry_time = row['date']
            entry_p_raw = row['ETH_Close']
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(entry_p_raw), True)
            
            atr_val = row['ETH_ATR_84'] if row['ETH_ATR_84'] > 0 else entry_p * 0.01
            real_sl_dist = min(atr_val * sl_mult, entry_p * 0.05)
            
            tp_price = entry_p + (atr_val * tp_mult)
            sl_price = entry_p - real_sl_dist
            
            max_hold_time = entry_time + pd.Timedelta(hours=hold_hours)
            future_candles = df_hour_indexed[entry_time:max_hold_time]
            
            if future_candles.empty: continue

            sl_hits = future_candles[future_candles['low'] <= sl_price]
            tp_hits = future_candles[future_candles['high'] >= tp_price]
            sl_hit_time = sl_hits.index.min() if not sl_hits.empty else pd.NaT
            tp_hit_time = tp_hits.index.min() if not tp_hits.empty else pd.NaT

            events = [('timeout', max_hold_time)]
            if pd.notna(sl_hit_time): events.append(('sl', sl_hit_time))
            if pd.notna(tp_hit_time): events.append(('tp', tp_hit_time))
            events.sort(key=lambda x: x[1])
            exit_reason, exit_time = events[0]
            
            exit_p = 0
            if exit_reason == 'sl': exit_p = self.env.apply_slippage(sl_price, False)
            elif exit_reason == 'tp': exit_p = self.env.apply_slippage(tp_price, False)
            else: exit_p = self.env.apply_slippage(future_candles.iloc[-1]['close'], False)

            potential_trades.append({
                'entry_time': entry_time, 'exit_time': exit_time,
                'entry_price': entry_p, 'exit_price': exit_p
            })
        
        if not potential_trades:
            return {'Return': 0, 'MDD': 0, 'Trades': 0, 'WinRate': 0, 'Params': ''}

        # --- 2. 통합 타임라인 생성 ---
        all_times = set()
        for trade in potential_trades:
            all_times.add(trade['entry_time'])
            all_times.add(trade['exit_time'])
        timeline = sorted(list(all_times))

        # --- 3. 시간순 시뮬레이션 (강제 쿨다운 적용) ---
        initial_equity = 1_000_000
        equity = initial_equity
        peak_value = initial_equity
        max_dd = 0
        wins = 0
        trades_executed = 0
        
        MAX_SLOTS = 3
        active_positions = [] 
        trade_queue = sorted(potential_trades, key=lambda x: x['entry_time'])
        
        # 🚨 [수정] 쿨다운 만료 시간 변수 (초기값: 아주 옛날)
        cooldown_until = pd.Timestamp("2000-01-01")

        for current_time in timeline:
            # A. 포지션 청산 (Exit)
            exited_this_step = [p for p in active_positions if p['exit_time'] == current_time]
            for pos in exited_this_step:
                sell_amt = pos['qty'] * pos['exit_price']
                exit_fee = sell_amt * self.env.FEE_RATE
                pos_krw = pos['qty'] * pos['entry_price']
                pnl = sell_amt - exit_fee - pos_krw
                equity += (pos_krw + pnl)
                if pnl > 0: wins += 1
                active_positions.remove(pos)

            # B. 포트폴리오 가치 및 MDD 체크
            unrealized_value = 0
            for pos in active_positions:
                try: market_price = df_hour_indexed.loc[:current_time].iloc[-1]['close']
                except: market_price = pos['entry_price']
                unrealized_value += pos['qty'] * market_price
            
            portfolio_value = equity + unrealized_value
            if portfolio_value > peak_value: peak_value = portfolio_value
            
            current_dd = (portfolio_value - peak_value) / peak_value * 100 if peak_value > 0 else 0
            if current_dd < max_dd: max_dd = current_dd

            # 🛡️ [핵심] MDD -10% 이하 발생 시 -> 15일간 쿨다운 설정
            # 단, 이미 쿨다운 중이면 시간 연장은 안 함 (너무 보수적일 수 있음)
            if current_dd < -10.0:
                if current_time >= cooldown_until: # 새로운 쿨다운 발동
                    cooldown_until = current_time + pd.Timedelta(days=15)

            # C. 포지션 진입 (Entry)
            while trade_queue and trade_queue[0]['entry_time'] == current_time:
                trade = trade_queue.pop(0)
                
                # 1. 쿨다운 체크: 현재 시간이 쿨다운 기간 내에 있으면 진입 스킵
                if current_time < cooldown_until:
                    continue 
                
                # 2. 슬롯 확인
                if len(active_positions) >= MAX_SLOTS: continue
                
                # 3. 자금 배정
                total_asset = equity + sum([p['qty'] * p['entry_price'] for p in active_positions])
                bet_amount = total_asset / MAX_SLOTS
                if equity < bet_amount: bet_amount = equity
                if bet_amount < 5000: continue
                
                qty = bet_amount / trade['entry_price']
                pos_krw = qty * trade['entry_price']
                equity -= pos_krw
                equity -= (pos_krw * self.env.FEE_RATE)
                
                active_positions.append({
                    'qty': qty, 'entry_price': trade['entry_price'],
                    'exit_time': trade['exit_time'], 'exit_price': trade['exit_price'],
                })
                trades_executed += 1
        
        final_ret = (portfolio_value - initial_equity) / initial_equity * 100
        win_rate = (wins / trades_executed * 100) if trades_executed > 0 else 0
        
        return {
            'Return': final_ret, 'MDD': max_dd, 'Trades': trades_executed, 'WinRate': win_rate,
            'Params': f"TP:{tp_mult} / SL:{sl_mult} / Hold:{hold_hours}H / Slots:{MAX_SLOTS}"
        }

def run_single_simulation(params, simulator, df_signals, df_hour):
    tp, sl, hold = params
    res = simulator.run_grid_test(df_signals, df_hour, tp, sl, hold)
    res.update({'TP': tp, 'SL': sl, 'Hold': hold})
    return res

# =========================================================
# 2. 메인 실행 블록
# =========================================================
if __name__ == "__main__":
    print("📂 Loading Data...")
    try:
        df_4h = pd.read_csv("eth_4hour_all.csv", parse_dates=['date'])
        df_hour = pd.read_csv("eth_hour.csv", parse_dates=['datetime'])
    except FileNotFoundError:
        print("❌ 파일을 찾을 수 없습니다.")
        exit()

    df_4h = df_4h[df_4h['date'] >= "2020-01-01"].copy()
    df_hour = df_hour[df_hour['datetime'] >= "2020-01-01"].copy()

    print("🔧 Data Cleaning & Alignment...")
    df_4h = df_4h.sort_values('date').drop_duplicates('date').reset_index(drop=True)
    df_hour = df_hour.sort_values('datetime').drop_duplicates('datetime').reset_index(drop=True)
    
    # 이상치 제거
    df_4h['pct_change'] = df_4h['ETH_Close'].pct_change()
    outlier_mask = (df_4h['pct_change'].abs() > 0.15)
    print(f"🗑️ Removed {outlier_mask.sum()} Outliers.")
    df_4h = df_4h[~outlier_mask].copy().reset_index(drop=True)
    
    print("⚙️ Calculating Indicators...")
    
    df_4h['ETH_Force_Index_13'] = (df_4h['ETH_Close'].diff(1) * df_4h['ETH_Volume']).ewm(span=13).mean()
    er_period = 10
    change = df_4h['ETH_Close'].diff(er_period).abs()
    volatility = df_4h['ETH_Close'].diff(1).abs().rolling(er_period).sum()
    df_4h['ETH_ER_10'] = change / (volatility + 1e-9) 
    
    cond_entry = ( (df_4h['ETH_RSI_84'] > 55) & (df_4h['ETH_Force_Index_13'] > 0) & (df_4h['ETH_ER_10'] > 0.20) )
    
    if 'BTC_Close' in df_4h.columns:
        df_4h['BTC_Ret_24h'] = df_4h['BTC_Close'].pct_change(6)
        df_4h['BTC_MA_20'] = df_4h['BTC_Close'].rolling(120).mean()
        cond_defense = ( (df_4h['BTC_Close'] > df_4h['BTC_MA_20']) & (df_4h['BTC_Ret_24h'] > -0.03) )
    else: cond_defense = True
        
    df_4h['Max_7d'] = df_4h['ETH_Close'].rolling(42).max()
    cond_dd = ((df_4h['ETH_Close'] / df_4h['Max_7d'] - 1) > -0.05)
    cond_sentiment = (df_4h['ETH_KP_Zscore'] > -2.0)

    df_4h['signal'] = 0
    df_4h.loc[(cond_entry & cond_defense & cond_dd & cond_sentiment), 'signal'] = 1
    
    df_signals = df_4h[df_4h['signal'] == 1].copy()
    print(f"✅ Signal Count (Refined): {len(df_signals)}")
    
    if df_signals.empty: print("❌ No signals found."); exit()

    sim = QuantSimulatorGrid()
    
    tp_list = [4.0, 5.0, 6.0]
    sl_list = [2.0, 2.5, 3.0]
    hold_list = [72, 96, 120]
    
    param_combinations = list(itertools.product(tp_list, sl_list, hold_list))
    num_cores = os.cpu_count() or 4
    
    print(f"🚀 Starting Parallel Grid Search on {num_cores} cores...")
    with Pool(processes=num_cores) as pool:
        runner = partial(run_single_simulation, simulator=sim, df_signals=df_signals, df_hour=df_hour)
        results = pool.map(runner, param_combinations)
    print("✅ Grid Search Completed!")

    df_res = pd.DataFrame(results)
    if not df_res.empty:
        best_ret = df_res.loc[df_res['Return'].idxmax()]
        df_safe = df_res[df_res['Return'] > 0]
        best_safe = df_safe.loc[df_safe['MDD'].idxmax()] if not df_safe.empty else None
        
        print("\n" + "="*60)
        print(f"🏆 [Revenue King]\nParams: {best_ret['Params']}\nReturn: {best_ret['Return']:.1f}% | MDD: {best_ret['MDD']:.2f}% | Trades: {best_ret['Trades']}")
        print("-" * 60)
        if best_safe is not None:
            print(f"🛡️ [Safety King]\nParams: {best_safe['Params']}\nReturn: {best_safe['Return']:.1f}% | MDD: {best_safe['MDD']:.2f}% | Trades: {best_safe['Trades']}")
        print("\n📊 Top 5 Combinations:")
        print(df_res.sort_values('Return', ascending=False).head(5)[['Params', 'Return', 'MDD', 'Trades']])


📂 Loading Data...
🔧 Data Cleaning & Alignment...
🗑️ Removed 3 Outliers.
⚙️ Calculating Indicators...
✅ Signal Count (Refined): 1643
🚀 Starting Parallel Grid Search on 88 cores...
✅ Grid Search Completed!

🏆 [Revenue King]
Params: TP:4.0 / SL:2.0 / Hold:72H / Slots:3
Return: 2664.3% | MDD: -71.41% | Trades: 26
------------------------------------------------------------
🛡️ [Safety King]
Params: TP:6.0 / SL:3.0 / Hold:72H / Slots:3
Return: 38.1% | MDD: -10.00% | Trades: 29

📊 Top 5 Combinations:
                                   Params   Return    MDD  Trades
0    TP:4.0 / SL:2.0 / Hold:72H / Slots:3 2,664.26 -71.41      26
9    TP:5.0 / SL:2.0 / Hold:72H / Slots:3 2,622.85 -71.43      24
10   TP:5.0 / SL:2.0 / Hold:96H / Slots:3 2,589.31 -71.50      32
20  TP:6.0 / SL:2.0 / Hold:120H / Slots:3 2,575.05 -71.55      30
18   TP:6.0 / SL:2.0 / Hold:72H / Slots:3 2,510.09 -71.47      28


In [42]:
# ... (위의 최종 시뮬레이터 코드, 즉 UpbitMicrostructure, QuantSimulatorGrid 클래스 정의 필요) ...

# 메인 실행 블록 (기존 코드 아래에 이어서 붙여넣기)
if __name__ == "__main__":
    # 1. 데이터 로드 (이미 되어 있다면 생략 가능)
    df_4h = pd.read_csv("eth_4hour_all.csv", parse_dates=['date'])
    df_hour = pd.read_csv("eth_hour.csv", parse_dates=['datetime'])
    
    # ... (데이터 정제 과정 생략, 이미 위에서 했으므로) ...
    
    # 2. MDD 추적용 단일 시뮬레이션 실행
    # (Revenue King 파라미터: TP:6.0 / SL:2.0 / Hold:120H / Slots:3)
    
    print("🕵️ MDD 발생 날짜 추적 중...")
    
    sim = QuantSimulatorGrid()
    
    # 내부 로직을 살짝 끄집어내서 상세 로그를 찍습니다.
    # (QuantSimulatorGrid 클래스 내부 코드를 그대로 가져와서 디버깅용으로 돌림)
    
    # 파라미터 설정
    tp_mult = 6.0
    sl_mult = 2.0
    hold_hours = 120
    MAX_SLOTS = 3
    
    # --- A. 거래 리스트 생성 ---
    potential_trades = []
    df_hour_indexed = df_hour.set_index('datetime').sort_index()

    for _, row in df_signals.iterrows():
        entry_time = row['date']
        entry_p_raw = row['ETH_Close']
        entry_p = sim.env.apply_slippage(sim.env.adjust_to_tick(entry_p_raw), True)
        
        atr_val = row['ETH_ATR_84'] if row['ETH_ATR_84'] > 0 else entry_p * 0.01
        real_sl_dist = min(atr_val * sl_mult, entry_p * 0.05)
        tp_price = entry_p + (atr_val * tp_mult)
        sl_price = entry_p - real_sl_dist
        
        max_hold_time = entry_time + pd.Timedelta(hours=hold_hours)
        future_candles = df_hour_indexed[entry_time:max_hold_time]
        
        if future_candles.empty: continue

        sl_hits = future_candles[future_candles['low'] <= sl_price]
        tp_hits = future_candles[future_candles['high'] >= tp_price]
        sl_hit_time = sl_hits.index.min() if not sl_hits.empty else pd.NaT
        tp_hit_time = tp_hits.index.min() if not tp_hits.empty else pd.NaT

        events = [('timeout', max_hold_time)]
        if pd.notna(sl_hit_time): events.append(('sl', sl_hit_time))
        if pd.notna(tp_hit_time): events.append(('tp', tp_hit_time))
        events.sort(key=lambda x: x[1])
        exit_reason, exit_time = events[0]
        
        exit_p = 0
        if exit_reason == 'sl': exit_p = sim.env.apply_slippage(sl_price, False)
        elif exit_reason == 'tp': exit_p = sim.env.apply_slippage(tp_price, False)
        else: exit_p = sim.env.apply_slippage(future_candles.iloc[-1]['close'], False)

        potential_trades.append({
            'entry_time': entry_time, 'exit_time': exit_time,
            'entry_price': entry_p, 'exit_price': exit_p, 'reason': exit_reason
        })
    
    # --- B. 타임라인 시뮬레이션 ---
    all_times = sorted(list(set([t['entry_time'] for t in potential_trades] + [t['exit_time'] for t in potential_trades])))
    
    initial_equity = 1_000_000
    equity = initial_equity
    peak_value = initial_equity
    active_positions = []
    trade_queue = sorted(potential_trades, key=lambda x: x['entry_time'])
    
    equity_curve = []

    for current_time in all_times:
        # 청산
        exited = [p for p in active_positions if p['exit_time'] == current_time]
        for pos in exited:
            sell_amt = pos['qty'] * pos['exit_price']
            exit_fee = sell_amt * sim.env.FEE_RATE
            pos_krw = pos['qty'] * pos['entry_price']
            equity += (pos_krw + (sell_amt - exit_fee - pos_krw))
            active_positions.remove(pos)

        # 진입
        while trade_queue and trade_queue[0]['entry_time'] == current_time:
            trade = trade_queue.pop(0)
            if len(active_positions) >= MAX_SLOTS: continue
            
            total_asset = equity + sum([p['qty'] * p['entry_price'] for p in active_positions])
            bet_amount = total_asset / MAX_SLOTS
            if equity < bet_amount: bet_amount = equity
            if bet_amount < 5000: continue
            
            qty = bet_amount / trade['entry_price']
            pos_krw = qty * trade['entry_price']
            equity -= pos_krw
            equity -= (pos_krw * sim.env.FEE_RATE)
            
            active_positions.append({
                'qty': qty, 'entry_price': trade['entry_price'],
                'exit_time': trade['exit_time'], 'exit_price': trade['exit_price']
            })

        # 평가금 계산
        unrealized_value = 0
        for pos in active_positions:
            try: 
                # 현재 시간의 종가 가져오기 (없으면 직전 종가)
                current_price = df_hour_indexed.loc[:current_time].iloc[-1]['close']
            except: 
                current_price = pos['entry_price']
            unrealized_value += pos['qty'] * current_price
        
        portfolio_value = equity + unrealized_value
        if portfolio_value > peak_value: peak_value = portfolio_value
        dd = (portfolio_value - peak_value) / peak_value * 100 if peak_value > 0 else 0
        
        equity_curve.append({
            'Date': current_time,
            'Equity': portfolio_value,
            'Peak': peak_value,
            'MDD': dd,
            'Positions': len(active_positions)
        })

    # --- C. 결과 출력 ---
    df_curve = pd.DataFrame(equity_curve)
    if df_curve.empty:
        print("거래 내역이 없습니다.")
    else:
        worst_day = df_curve.loc[df_curve['MDD'].idxmin()]
        print("\n" + "="*60)
        print(f"🔥 [MDD 확정 날짜] : {worst_day['Date']}")
        print(f"📉 그날의 MDD : {worst_day['MDD']:.2f}%")
        print(f"💰 그날의 자산 : {worst_day['Equity']:,.0f}원 (고점: {worst_day['Peak']:,.0f}원)")
        print(f"🎰 보유 포지션 수 : {worst_day['Positions']}개")
        print("="*60)
        
        print("\n[그 날 전후 5일간의 기록]")
        start_view = worst_day['Date'] - pd.Timedelta(days=5)
        end_view = worst_day['Date'] + pd.Timedelta(days=5)
        print(df_curve[(df_curve['Date'] >= start_view) & (df_curve['Date'] <= end_view)])


🕵️ MDD 발생 날짜 추적 중...

🔥 [MDD 확정 날짜] : 2023-01-09 17:00:00
📉 그날의 MDD : -71.60%
💰 그날의 자산 : 7,942,504원 (고점: 27,965,247원)
🎰 보유 포지션 수 : 3개

[그 날 전후 5일간의 기록]
                    Date       Equity          Peak    MDD  Positions
1175 2023-01-09 17:00:00 7,942,504.20 27,965,247.05 -71.60          3
1176 2023-01-09 21:00:00 8,009,856.05 27,965,247.05 -71.36          3
1177 2023-01-10 01:00:00 8,079,613.32 27,965,247.05 -71.11          3
1178 2023-01-10 13:00:00 8,017,072.32 27,965,247.05 -71.33          3
1179 2023-01-10 17:00:00 8,057,964.51 27,965,247.05 -71.19          3
1180 2023-01-10 21:00:00 8,024,288.59 27,965,247.05 -71.31          3
1181 2023-01-11 01:00:00 8,082,018.75 27,965,247.05 -71.10          3
1182 2023-01-11 05:00:00 8,108,478.40 27,965,247.05 -71.01          3
1183 2023-01-12 01:00:00 8,048,342.82 27,965,247.05 -71.22          3
1184 2023-01-12 05:00:00 8,103,667.55 27,965,247.05 -71.02          3
1185 2023-01-12 09:00:00 8,507,778.65 27,965,247.05 -69.58          3
1186 202